# Downscaling MODIS LST Imagery from 1000m to 10m Resolution Using Sentinel-2 Data

**Author:** Mitch Porter  
**Purpose:** Application for Project Drawdown  
**Usage:** This notebook enables the downscaling of 1000m MODIS Land Surface Temperature (LST) imagery to 10m resolution using Sentinel-2 NDVI data and linear regression. The entire process is cloud-based, leveraging Google Earth Engine to analyze and visualize LST at any location and time period where data is available.                                
**References:** 1. Huryna, H., Cohen, Y., Karnieli, A., Panov, N., Kustas, W. P., Agam, N. (2019). Evaluation of TsHARP Utility for Thermal Sharpening of Sentinel-3 Satellite Images Using Sentinel-2 Visual Imagery. Remote Sensing, 11, 2304. 2. https://towardsdatascience.com/downscaling-a-satellite-thermal-image-from-1000-m-to-10-m-python-3b2ed19ff103

---
## How to Use This Script

1. **Select the Area of Interest (AOI):** Use the interactive map to draw a rectangle over your desired study area.
2. **Choose Date Range:** Select the start and end dates to filter the available MODIS and Sentinel-2 imagery within that period.
3. **Retrieve Data:** Click the "Get Data" button to fetch matching MODIS and Sentinel-2 image pairs that were captured within 12 hours of each other.
4. **Run Analysis:** After selecting the desired date from the dropdown, click "Run Analysis" to:
   - Calculate NDVI from Sentinel-2 imagery.
   - Downscale MODIS LST to 10m resolution using the calculated NDVI.
   - Visualize the NDVI, original MODIS LST, and downscaled LST images.
   - Plot the linear regression used for downscaling.
5. **Change Date (Optional):** If multiple image pairs are available, select a different date from the dropdown and click 'Run Analysis'

**Note:** This notebook requires access to Google Earth Engine and works entirely from the cloud


In [ ]:
# Install required packages
!pip install geemap leafmap ipywidgets matplotlib ipyleaflet

In [ ]:
import ee
import leafmap
import ipyleaflet
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import urllib.request
from PIL import Image
import numpy as np

# Iinitialize EE and Trigger EE authentication if needed
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# Define Helper Functions. I would normally put all these in a utils folder and import using 'from utils import *', 
# but for the simplicity of this code example and job application I chose to put them in the notebook

def apply_cloud_mask(image):
    """
    Apply cloud mask to remove clouds from images
    """
    band_names = image.bandNames()
    cloud_mask = ee.Algorithms.If(
        band_names.contains('QA60'),
        image.select('QA60').bitwiseAnd(1 << 10).eq(0),
        image.select('MSK_CLASSI_OPAQUE').eq(0).And(image.select('MSK_CLASSI_CIRRUS').eq(0))
    )
    return image.updateMask(ee.Image(cloud_mask))

def filter_clouds(image_collection, max_cloud_percentage=25):
    """
    Remove images with cloud cover greater than the specified percentage.
    """
    return image_collection.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_percentage))

def calculate_ndvi(image):
    """
    Calculate NDVI as a new image.
    """
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return ndvi

def download_data(aoi, start_date, end_date):
    """
    Download S2 and MODIS data from GEE
    """
    aoi_geometry = ee.Geometry.Polygon(aoi)
    
    modis_collection = ee.ImageCollection('MODIS/061/MOD11A1').filterBounds(aoi_geometry).filterDate(start_date, end_date)
    s2_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(aoi_geometry).filterDate(start_date, end_date)
    s2_collection = filter_clouds(s2_collection).map(apply_cloud_mask)
    
    total_modis_images = modis_collection.size().getInfo()
    total_s2_images = s2_collection.size().getInfo()
    
    # Filter MODIS and Sentinel-2 images that are taken within 8 hours of each other to ensure more accurate downscaling
    combined_images = []
    for s2_img in s2_collection.getInfo()['features']:
        s2_date = datetime.utcfromtimestamp(s2_img['properties']['system:time_start'] / 1000)
        for modis_img in modis_collection.getInfo()['features']:
            modis_date = datetime.utcfromtimestamp(modis_img['properties']['system:time_start'] / 1000)
            if abs((modis_date - s2_date).total_seconds()) <= 12 * 3600:
                combined_images.append({
                    'date': s2_date,
                    'modis_image': ee.Image(modis_img['id']),
                    's2_image': ee.Image(s2_img['id'])
                })
    
    return total_modis_images, total_s2_images, combined_images

def downscale_thermal(modis_image, ndvi_image, aoi_geometry):
    """
    Downscale MODIS imagery from 1000m to 10m
    """
    # Rescale MODIS LST to match Sentinel-2 resolution
    modis_rescaled = modis_image.select('LST_Day_1km').reproject(crs=ndvi_image.projection(), scale=10)
    
    # Create a combined image with only NDVI and LST_Day_1km bands
    combined = ndvi_image.addBands(modis_rescaled).select(['NDVI', 'LST_Day_1km'])
    
    # Linear regression model with NDVI and LST_Day_1km
    regression = combined.reduceRegion(
        reducer=ee.Reducer.linearFit(),
        geometry=aoi_geometry,
        scale=1000,
        maxPixels=1e8
    )
    
    slope = ee.Number(regression.get('scale'))
    intercept = ee.Number(regression.get('offset'))
    
    # Downscale MODIS temperature and apply geometry
    downscaled = ndvi_image.multiply(slope).add(intercept).clip(aoi_geometry)
    return downscaled.rename('Downscaled_LST')

    
def visualize_image(image, title, aoi_geometry, band_name='NDVI', unit='', date=None):
    """
    Visualizes S2, MODIS 1000m, and MODIS 10m downscaled images
    """
    # Clip the image to the area of interest
    image_clipped = image.clip(aoi_geometry)
    
    # Convert MODIS LST to Kelvin
    if band_name == 'LST_Day_1km' or band_name == 'Downscaled_LST':
        image_clipped = image_clipped.multiply(0.02)
    
    # Compute the min, max, and standard deviation
    stats = image_clipped.reduceRegion(
        reducer=ee.Reducer.minMax().combine(reducer2=ee.Reducer.stdDev(), sharedInputs=True),
        geometry=aoi_geometry,
        scale=10,
        maxPixels=1e8
    ).getInfo()
    
    min_val = stats[f'{band_name}_min']
    max_val = stats[f'{band_name}_max']
    
    # Generate the thumbnail URL with the computed min and max values
    url = image_clipped.getThumbURL({
        'min': min_val,
        'max': max_val,
        'dimensions': 512,
        'region': aoi_geometry,
        'palette': ['blue', 'yellow', 'red']
    })
    
    # Open the URL and read the image
    with urllib.request.urlopen(url) as url_open:
        img = Image.open(url_open)
        img_array = np.array(img)
    
    # Display the image using matplotlib
    plt.figure(figsize=(10, 10))
    im = plt.imshow(img_array, cmap='jet', vmin=min_val, vmax=max_val)
    
    # Add dates to the titles
    if date:
        plt.title(f"{title} - {date}")
    else:
        plt.title(title)
    
    # Place the colorbar below the image and make it smaller
    cbar = plt.colorbar(im, orientation='horizontal', fraction=0.046, pad=0.04)
    if band_name != 'NDVI':  # Only show the unit if it's not NDVI
        cbar.set_label(unit)
    
    plt.axis('off')
    plt.show()

def plot_regression(ndvi_image, modis_image, aoi_geometry):
    """
    Plots the linear regression between NDVI and MODIS LST values.
    """
    # Extract values for NDVI and MODIS LST
    combined = ndvi_image.addBands(modis_image.select('LST_Day_1km')).reduceRegion(
        reducer=ee.Reducer.toList(),
        geometry=aoi_geometry,
        scale=10,
        maxPixels=1e8
    )
    
    # Get the lists of NDVI and LST values
    ndvi_values = combined.get('NDVI').getInfo()
    lst_values = combined.get('LST_Day_1km').getInfo()
    
    # Ensure the lists are not empty
    if ndvi_values is None or lst_values is None or len(ndvi_values) == 0 or len(lst_values) == 0:
        print("No valid data for regression plot.")
        return
    
    # Convert lists to NumPy arrays
    ndvi_values = np.array(ndvi_values)
    lst_values = np.array(lst_values)

    # Convert LST values to Kelvin
    lst_values = lst_values * 0.02
    
    # Perform linear regression
    slope, intercept = np.polyfit(ndvi_values, lst_values, 1)
    
    # Plot the scatter plot with the regression line
    plt.figure(figsize=(10, 6))
    plt.scatter(ndvi_values, lst_values, label='Data points')
    plt.plot(ndvi_values, slope * ndvi_values + intercept, color='red', label=f'Linear fit: y={slope:.2f}x + {intercept:.2f}')
    plt.title('Linear Regression between NDVI and MODIS LST')
    plt.xlabel('NDVI')
    plt.ylabel('MODIS LST (K)')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Create the Interactive Map and Widgets
m = leafmap.Map(center=[34.0522, -118.2437], zoom=13, basemap='Google Hybrid')

# Add drawing control using ipyleaflet
draw_control = ipyleaflet.DrawControl(
    rectangle={"shapeOptions": {"color": "#0000FF"}},
    polygon={"shapeOptions": {"color": "#0000FF"}},
    polyline={},
    circle={},
    circlemarker={},
    edit=False,  # Disable editing
    remove=False
)

m.add_control(draw_control)

# Widget definitions
start_date_picker = widgets.DatePicker(description='Start Date', disabled=False)
end_date_picker = widgets.DatePicker(description='End Date', disabled=False)
get_data_button = widgets.Button(description="Get Data", button_style='info')
image_date_picker = widgets.Dropdown(description='Select Date:', disabled=True)
run_button = widgets.Button(description="Run Analysis", button_style='success', disabled=True)

# Create an output widget for displaying figures
output_area = widgets.Output()

def on_get_data_click(b):
    if not draw_control.last_draw:
        print("Please draw an AOI on the map before getting data.")
        return
    
    aoi = draw_control.last_draw['geometry']['coordinates'][0]
    global aoi_geometry  # Store the AOI globally for visualization
    aoi_geometry = ee.Geometry.Polygon(aoi)
    start_date = start_date_picker.value.strftime('%Y-%m-%d')
    end_date = end_date_picker.value.strftime('%Y-%m-%d')
    
    total_modis_images, total_s2_images, combined_images = download_data(aoi, start_date, end_date)
    
    print(f"Total Sentinel-2 images: {total_s2_images}")
    print(f"Total MODIS images: {total_modis_images}")
    
    if combined_images:
        num_images = len(combined_images)
        print(f"Found {num_images} matching image sets.")
        
        dates = [img_set['date'].strftime('%Y-%m-%d %H:%M') for img_set in combined_images]
        image_date_picker.options = dates
        image_date_picker.disabled = False
        run_button.disabled = False
        
        def on_run_button_click(b):
            selected_date = image_date_picker.value
            selected_img_set = next(item for item in combined_images if item['date'].strftime('%Y-%m-%d %H:%M') == selected_date)
            
            # Clear only the figure output area
            with output_area:
                clear_output(wait=True)
                
                modis_image = selected_img_set['modis_image']
                s2_image = selected_img_set['s2_image']
                
                # Calculate NDVI as a new image
                ndvi_image = calculate_ndvi(s2_image)
                
                # Downscale thermal data using Earth Engine and apply geometry
                downscaled_image = downscale_thermal(modis_image, ndvi_image, aoi_geometry)
                
                # Visualize images
                visualize_image(ndvi_image, "Sentinel-2 NDVI", aoi_geometry, band_name='NDVI', date=selected_date)
                visualize_image(modis_image.select('LST_Day_1km'), "MODIS LST", aoi_geometry, band_name='LST_Day_1km', unit='K', date=selected_date)
                visualize_image(downscaled_image, "Downscaled Temperature", aoi_geometry, band_name='Downscaled_LST', unit='K', date=selected_date)
                
                print('Wait for the regression figure to display, it can take a while :)')
                
                # Plot the linear regression between NDVI and MODIS LST
                plot_regression(ndvi_image, modis_image, aoi_geometry)
        
        run_button.on_click(on_run_button_click)
    else:
        print("No matching MODIS and Sentinel-2 images found within the selected date range.")

# Connect the data button to the get data function
get_data_button.on_click(on_get_data_click)

# Display the map, widgets, and output area
display(m, start_date_picker, end_date_picker, get_data_button, image_date_picker, run_button, output_area)